In [7]:
# load model

import cobra

pheast = cobra.io.read_sbml_model("./data/iLC915_rewritten.xml")

In [16]:
# run the model/optimizes for cell growth

pheast.optimize()
pheast.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1293,EX_m1293,1,0,0.00%
m1298,EX_m1298,0.7968,0,0.00%
m1300,EX_m1300,0.4628,0,0.00%
m1301,EX_m1301,0.03308,0,0.00%
m1304,EX_m1304,0.00246,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.0803,0,0.00%
m1291,EX_m1291,-1.597,0,0.00%
m1349,EX_m1349,-3.432,0,0.00%


In [69]:
# find glucose reactions (it can be seen that it is the carbon source being taken up at 1 mmol gDW^-1 h^-1)
pheast.metabolites.m1293.reactions

frozenset({<Reaction EX_m1293 at 0x7f4664e51850>,
           <Reaction r1145 at 0x7f46647c6430>})

In [53]:
# exchange reaction in general

pheast.reactions.EX_m1293


Reaction identifier,EX_m1293
Name,EX_m1293
Memory address,0x07f4664e51850
Stoichiometry,m1293 <=> alpha-D-Glucose_C6H12O6 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [52]:
# this reaction seems to be the uptake from medium (it is defined in this model as boundary metabolites -> extracellular --> cytosolic
# and we have to define the medium boudary at reaction from boundary to extracellular)

pheast.reactions.r1145


Reaction identifier,r1145
Name,Uptake of alpha-D-Glucose
Memory address,0x07f46647c6430
Stoichiometry,m1293 --> m2 alpha-D-Glucose_C6H12O6 --> alpha-D-Glucose_C6H12O6
GPR,
Lower bound,0
Upper bound,0


In [43]:
# look for methanol

pheast.reactions.query("methanol", "name")

[<Reaction r1158 at 0x7f46647d7d30>]

In [21]:
# this is methanol uptake

pheast.reactions.r1145

Reaction identifier,r1145
Name,Uptake of alpha-D-Glucose
Memory address,0x07f46647c6430
Stoichiometry,m1293 --> m2 alpha-D-Glucose_C6H12O6 --> alpha-D-Glucose_C6H12O6
GPR,
Lower bound,0.0
Upper bound,1.0


In [22]:
pheast.reactions.r1145.bounds = 0, 0  # no glucose 
pheast.reactions.r1158.bounds = 0, 1  # methanol at mmol gDW^-1 h^-1

In [23]:
# it grows on methanol

pheast.optimize()
pheast.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1297,EX_m1297,1,0,0.00%
m1298,EX_m1298,1.258,0,0.00%
m1300,EX_m1300,0.06602,0,0.00%
m1301,EX_m1301,0.004719,0,0.00%
m1304,EX_m1304,0.0003508,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.01145,0,0.00%
m1291,EX_m1291,-0.372,0,0.00%
m1349,EX_m1349,-1.643,0,0.00%


In [25]:
pheast.reactions.query("methane", "name")

[]

In [81]:
methane = pheast.add_metabolites([cobra.Metabolite(
    'boundary_methane',
    formula='C1H4',
    name='bou_methane',
    compartment='C_b'),
                                cobra.Metabolite(
    'extracellular methane',
    formula='C1H4',
    name='ex_methane',
    compartment='C_e'),
                                cobra.Metabolite(
    'cytosolic_methane',
    formula='C1H4',
    name='cyt_methane',
    compartment='C_c')])

In [83]:
pheast.metabolites.boundary_methane

Metabolite identifier,boundary_methane
Name,methane
Memory address,0x07f4664055d90
Formula,C1H4
Compartment,C_b
In 0 reaction(s),


In [26]:
pheast.compartments

{'C_c': 'cytosol',
 'C_e': 'extracellular',
 'C_m': 'mitochondrion',
 'C_p': 'peroxisome',
 'C_v': 'vacuole',
 'C_g': 'golgi',
 'C_r': 'ER',
 'C_b': 'boundary'}

In [63]:
method_list = [method for method in dir(pheast) if method.startswith('__') is False]
print(method_list)

['_annotation', '_compartments', '_contexts', '_id', '_populate_solver', '_repr_html_', '_sbml', '_set_id_with_model', '_solver', '_tolerance', '_trimmed', '_trimmed_genes', '_trimmed_reactions', 'add_boundary', 'add_cons_vars', 'add_groups', 'add_metabolites', 'add_reaction', 'add_reactions', 'annotation', 'boundary', 'compartments', 'constraints', 'copy', 'demands', 'description', 'exchanges', 'genes', 'get_associated_groups', 'get_metabolite_compartments', 'groups', 'id', 'medium', 'merge', 'metabolites', 'name', 'notes', 'objective', 'objective_direction', 'optimize', 'problem', 'reactions', 'remove_cons_vars', 'remove_groups', 'remove_metabolites', 'remove_reactions', 'repair', 'sinks', 'slim_optimize', 'solver', 'summary', 'tolerance', 'variables']
